Bag of words (количество встреч слов в тексте)   
TF-IDF (нормируем по тому, как они важны)    
TF = (Frequency of the word in the sentence) / (Total number of words in the sentence)    
IDF = (Total number of sentences (documents))/(Number of sentences (documents) containing the word)


# Загрузка и проверка данных



In [2]:
import re
import pandas as pd
import numpy as np

In [3]:
text = pd.read_csv('/content/inetensive_1.csv')
text.head()

,Unnamed: 0,text,label
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,4,"Oil prices soar to all-time record, posing new...",2


In [4]:
len(text)

120000

In [5]:
# reg_ex = r'[\\!"#$%&\'()*+,/:;.<=>\?@[\]^_\-`{|}\n\t]'

# Preprocessing

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import numpy as np
from nltk.tokenize import  word_tokenize


#Preprocessing the text data
sentences = []
word_set = []

for i in range(len(text)):
    x = [i.lower() for  i in word_tokenize(text['text'][i]) if i.isalpha()]
    sentences.append(x)


In [9]:
sentences[0]

['wall',
 'bears',
 'claw',
 'back',
 'into',
 'the',
 'black',
 'reuters',
 'reuters',
 'wall',
 'street',
 'of',
 'are',
 'seeing',
 'green',
 'again']

In [10]:
from tqdm import tqdm

# Разобъем слова на n-грамы по 2 символа (на окончании 1)

In [172]:
for sent in sentences[0]:
  print(sent)
  for j in range(len(sent)):
    print(sent[j:j+2])
  break

wall
wa
al
ll
l


In [21]:
def generate_ngrams(texts, SymToCombine=2):
     l = len(texts)
     out_sent = []
     for i in tqdm(range(l)):
      words = texts[i]
      out_word = []
      for word in words:
        output = []
        for j in range(len(word)):
          output.append(word[j:j+SymToCombine])
        out_word.append(output)
      out_sent.append(out_word)
     return out_sent


ngrams = generate_ngrams(texts=sentences, SymToCombine=2)

100%|██████████| 120000/120000 [00:13<00:00, 8644.12it/s] 


In [22]:
ngrams[0]

[['wa', 'al', 'll', 'l'],
 ['be', 'ea', 'ar', 'rs', 's'],
 ['cl', 'la', 'aw', 'w'],
 ['ba', 'ac', 'ck', 'k'],
 ['in', 'nt', 'to', 'o'],
 ['th', 'he', 'e'],
 ['bl', 'la', 'ac', 'ck', 'k'],
 ['re', 'eu', 'ut', 'te', 'er', 'rs', 's'],
 ['re', 'eu', 'ut', 'te', 'er', 'rs', 's'],
 ['wa', 'al', 'll', 'l'],
 ['st', 'tr', 're', 'ee', 'et', 't'],
 ['of', 'f'],
 ['ar', 're', 'e'],
 ['se', 'ee', 'ei', 'in', 'ng', 'g'],
 ['gr', 're', 'ee', 'en', 'n'],
 ['ag', 'ga', 'ai', 'in', 'n']]

## Множество всех n-грам

In [24]:
ngram_set = []
for sent in tqdm(ngrams):
  for word in sent:
    for ngram in word:
      if ngram not in ngram_set:
        ngram_set.append(ngram)

100%|██████████| 120000/120000 [00:23<00:00, 5127.88it/s]


In [26]:
ngram_set = set(ngram_set)
total_documents = len(text)
print(total_documents)

index_dict = {}  # словарь с индексами
i = 0
for ngram in ngram_set:
    index_dict[ngram] = i
    i += 1
print(len(index_dict))

120000
690


## Подсчёт частоты каждой n-грамы в тексте

In [54]:
# for sent in ngrams_:
#   for word in sent:
#     for ngram in word:
def count_dict(sentences=ngrams):
    ngram_count = {}

    for ngram in tqdm(ngram_set):
        ngram_count[ngram] = 0
        for sent in (sentences):
          for i in range(len(sent)):
            if ngram in sent[i]:
              ngram_count[ngram] += 1
    return ngram_count

nrgam_count = count_dict(sentences=ngrams)

100%|██████████| 690/690 [05:10<00:00,  2.22it/s]


## TF

In [99]:
# TF = (Frequency of the word in the sentence) / (Total number of words in the sentence) - Term Frequence
def termfreq(document, ngram):
    occurance = 0
    N = 0
    for words in document:
      N += (len([ng for ng in words]))
      occurance += len([word for word in words if word == ngram])
    return occurance/N

In [101]:
termfreq(ngrams[0], 'wa')

0.02702702702702703

## IDF

In [121]:
# IDF = (Total number of sentences (documents))/(Number of sentences (documents) containing the word) - Inverse Document Frequency
def inverse_doc_freq(ngram):
    try:
        ngram_occurance = nrgam_count[ngram] + 1
    except:
        ngram_occurance = 1
    return np.log(total_documents/ngram_occurance)

## Получаем эмбеддинги текстов с помощью tf-idf

In [129]:
def tf_idf(ngrams_sentence):
    tf_idf_vec = np.zeros((len(ngram_set),))
    for word in ngrams_sentence:
      for ng in word:
        tf = termfreq(ngrams_sentence, ng)
        idf = inverse_doc_freq(ng)
        value = tf*idf
        tf_idf_vec[index_dict[ng]] = value
    return tf_idf_vec

In [130]:
#TF-IDF Encoded text, длина вектора = количество n-gram в словаре
vectors = []

for i in tqdm(range(len(ngrams))):
    vec = tf_idf(ngrams[i])
    vectors.append(vec)

100%|██████████| 120000/120000 [08:29<00:00, 235.71it/s]


# Обучим логистическую регрессию для проверки

In [135]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [138]:
X_train, X_test, y_train, y_test = train_test_split(
    vectors, text['label'], test_size = 0.3, random_state=42
)


logit = LogisticRegression(random_state=42)

In [139]:
logit.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [140]:
y_pred_proba = logit.predict_proba(X_test)

In [160]:
y_pred = logit.predict(X_test)

In [162]:
accuracy_score(y_test, y_pred)

0.7004444444444444

In [168]:
roc_auc_score(y_test, y_pred_proba, multi_class='ovr')

0.8911401736212246